In [1]:
import plotly.express as px
import plotly.io as pio
import pandas as pd
import numpy as np
pio.renderers.default='iframe'
pd.options.mode.chained_assignment = None  

df = pd.read_csv('asoiaf_nodes_prop.csv')

In [2]:
def prepare_buckets(subset_df, column_name, level):
    mean = subset_df[column_name].mean()
    std = subset_df[column_name].std()
    if str(std)=='nan': std = 0

        # Iterate over all rows and calculate buckets
    for index, row in subset_df.iterrows():
        i = 0
        j = 0
        flag = True
        while flag:
            # left
            left_start = mean - (i * std)
            left_end = mean - ((i-1) * std)
            #right
            right_start = mean - (j * std)
            right_end = mean - ((j-1) * std)
            
            # Condition checking
            if left_start<=row[column_name]<=left_end:
                subset_df.loc[index, 'bucket_'+str(level)] = str("("+str(left_start)+","+str(left_end)+")")
                flag=False
            if right_start<=row[column_name]<=right_end:
                subset_df.loc[index, 'bucket_'+str(level)] = str("("+str(right_start)+","+str(right_end)+")")
                flag=False
            
            # Update pointer
            i+=1
            j-=1
    return subset_df

## Degree

In [3]:
df = prepare_buckets(df, 'degree', 1)

## Peel, Pagerank, Diversity, and Betweenness

In [4]:
level = 2
features = ['peel', 'pagerank', 'diversity', 'betweenness']

for feature in features:
    df1 = pd.DataFrame(data = None, columns= df.columns)
    buckets = df['bucket_'+str(level-1)].unique()
    for bucket in buckets:
        pass_df = df[df['bucket_'+str(level-1)]==bucket]
        df1 = df1.append(prepare_buckets(pass_df, feature, level), ignore_index=False) 
    df=df1
    level+=1

## Treemap

In [5]:
import plotly.express as px
fig = px.treemap(df1, path=[px.Constant("all"), 'bucket_1', 'bucket_2', 'bucket_3', 'bucket_4', 'bucket_5'])
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [6]:
fig.write_image("HW1_jc2777_treemap.png")